In [ ]:
#
# This cell demonstrates a Flask Application that makes 3 Legged-Oauth calls to Blackboard Learn.
# This is meant for demonstration purposes only. To be complete you would cache the access token,
# and have session management for each user that is using the web application.
from flask import Flask, redirect, request, session, url_for

try:
    import Config
except ModuleNotFoundError:
    print("Config module not found. Did you copy Config_template.py to Config.py and set values?")

import requests

# Configuration
KEY = Config.config['key'] #Learn doc calls this a key. Industry may ref as CLIENT_ID
SECRET = Config.config['secret']
BLACKBOARD_DOMAIN = Config.config['blackboard_domain']
THIS_DOMAIN = Config.config['this_domain']
REDIRECT_URI = f'https://{THIS_DOMAIN}/callback'
AUTH_BASE_URL = f'https://{BLACKBOARD_DOMAIN}/learn/api/public/v1/oauth2/authorizationcode'
TOKEN_URL = f'https://{BLACKBOARD_DOMAIN}/learn/api/public/v1/oauth2/token'
A_USER_URL = f'https://{BLACKBOARD_DOMAIN}/learn/api/public/v1/users/userName:astudent'

# Import the 'datetime' module to work with date and time
import datetime
# Get the current date and time
now = datetime.datetime.now()
# Create a datetime object representing the current date and time
# Display a message indicating what is being printed
print("Current date and time : ")
# Print the current date and time in a specific format
print(now.strftime("%Y-%m-%d %H:%M:%S"))

""" We don't do any 2LO in this notebook. This code is here for reference.
def get_access_token(key, secret):
    #Authenticate with Blackboard Learn REST API using 2-legged OAuth.
    #grant_type is client_credentials.
    payload = {
        'grant_type': 'client_credentials',
    }
    response = requests.post(TOKEN_URL, auth=(key, secret), data=payload)
    response.raise_for_status()  # Raises stored HTTPError, if one occurred.
    
    print("Authentication successful.")
    return response.json().get('access_token')
"""

def get_user(access_token, user_url):
    """Make a REST API call to get a user."""
    headers = {
        'Authorization': f'Bearer {access_token}',
    }
    response = requests.get(user_url, headers=headers)
    response.raise_for_status()  # Raises stored HTTPError, if one occurred.
    
    return response.json()
    
app = Flask(__name__)
app.secret_key = 'n0w1sth3t0me!'  # Change this to a random secret key

@app.route('/')
def hello_world():
    return 'Hello there, 3LO world!'

@app.route('/auser')
def auser():
    try:
        access_token = session.get('access_token')
        if not access_token:
            return 'Access token is missing. Please log in again.'
        
        # Use access token to get user's courses
        user = get_user(access_token, A_USER_URL)
        
        print("A user fetched successfully.")
        return(user)  # You might want to format this output or handle it differently depending on your needs
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
    except Exception as err:
        print(f"An error occurred: {err}")

@app.route('/login')
def login():
    """Redirect user to Blackboard authorization page."""
    auth_url = f"{AUTH_BASE_URL}?response_type=code&client_id={KEY}&redirect_uri={REDIRECT_URI}"
    return redirect(auth_url)

@app.route('/callback')
def callback():
    """Exchange code for a token and fetch user's courses."""
    code = request.args.get('code')
    token_data = exchange_code_for_token(code)
    session['access_token'] = token_data.get('access_token')
    return redirect(url_for('auser'))

def exchange_code_for_token(code):
    #Exchange authorization code for access token.
    #grant_type is authorization_code
    payload = {
        'grant_type': 'authorization_code',
        'code': code,
        'redirect_uri': REDIRECT_URI,
        'client_id': KEY,
        'client_secret': SECRET
    }
    response = requests.post(TOKEN_URL, auth=(KEY, SECRET), data=payload)
    print(response.json())
    return response.json()

if __name__ == '__main__':
    app.run(port=5310)
